In [2]:
import os
from letta_client import Letta

# Configuration
CONFIG = {
    "name": "benchmark_agent",
    "llm_config": {
        "model": "openai/gpt-oss-20b",
        "model_endpoint_type": "openai",
        "model_endpoint": "https://openrouter.ai/api/v1",
        "context_window": 16000  # CRITICAL: Small enough to trigger memory management
    },
    "embedding_config": {
        "embedding_model": "openai/text-embedding-3-large", 
        "embedding_endpoint_type": "openai",
        "embedding_endpoint": "https://openrouter.ai/api/v1",
        "embedding_dim": 1536
    },
    "memory_blocks": [
        {
            "label": "human", 
            "value": "The user is participating in a benchmark evaluation."
        },
        {
            "label": "persona", 
            "value": "I am an AI assistant being evaluated for my memory management capabilities."
        }
    ]
}

# Connect to local server
client = Letta(
    base_url="http://localhost:8283", 
    timeout=1000, 
    )

# Create agent
agent = client.agents.create(**CONFIG)
print(f"✓ Agent created: {agent.id}")

✓ Agent created: agent-37d53d3e-03c2-4b65-b368-5f28f658433f


In [3]:
response = client.agents.messages.create(
    agent_id=agent.id,
    messages=[{"role": "user", "content": [{"type": "text", "text": "Hello"}]}]
)

In [4]:
response

LettaResponse(messages=[ReasoningMessage(id='message-ae054eb7-c027-4e22-9a07-1affc5624ba5', date=datetime.datetime(2025, 12, 22, 13, 44, 39, tzinfo=TzInfo(0)), name=None, message_type='reasoning_message', otid='ae054eb7-c027-4e22-9a07-1affc5624b80', sender_id=None, step_id='step-d9f2f5f6-8f72-4a77-9522-6f93ab393b6b', is_err=None, seq_id=None, run_id='run-454276dd-a266-4c15-bc15-53c7ab6da151', source='non_reasoner_model', reasoning='User greeted. Offer friendly chat. Avoid generic prompts. Keep persona.', signature=None), AssistantMessage(id='message-ae054eb7-c027-4e22-9a07-1affc5624ba5', date=datetime.datetime(2025, 12, 22, 13, 44, 39, tzinfo=TzInfo(0)), name=None, message_type='assistant_message', otid='ae054eb7-c027-4e22-9a07-1affc5624b81', sender_id=None, step_id='step-d9f2f5f6-8f72-4a77-9522-6f93ab393b6b', is_err=None, seq_id=None, run_id='run-454276dd-a266-4c15-bc15-53c7ab6da151', content='Hey there! Great to see you. How’s your day going?')], stop_reason=LettaStopReason(message_t

In [8]:
from letta_client import Letta
import json

client = Letta(base_url="http://localhost:8283")

# Create agent
# agent = client.agents.create(
#     name="inspect_test",
#     llm_config={...},
#     embedding_config={...},
#     memory_blocks=[
#         {"label": "human", "value": "Test user"},
#         {"label": "persona", "value": "Test assistant"}
#     ]
# )

# Inspect agent configuration
agent_details = client.agents.get(agent.id)

print("="*60)
print("LETTA AGENT CONFIGURATION")
print("="*60)

# Check if there's a system prompt field
print("\n1️⃣ Agent System Prompt:")
if hasattr(agent_details, 'system'):
    print(agent_details.system)
elif hasattr(agent_details, 'system_prompt'):
    print(agent_details.system_prompt)
else:
    print("⚠️  No explicit system prompt field found")

print("\n2️⃣ Memory Blocks:")
if hasattr(agent_details, 'memory'):
    for block in agent_details.memory.blocks:
        print(f"\n[{block.label}]")
        print(block.value)

print("\n3️⃣ Available Tools:")
if hasattr(agent_details, 'tools'):
    for tool in agent_details.tools:
        print(f"  - {tool}")

print("\n4️⃣ Full Agent State (JSON):")
print(json.dumps(agent_details.model_dump(), indent=2))

# Cleanup
client.agents.delete(agent.id)

AttributeError: 'AgentsClient' object has no attribute 'get'

In [9]:
import logging
import httpx

# Enable httpx detailed logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("httpx")
logger.setLevel(logging.DEBUG)

# Also add a custom handler to capture request bodies
class RequestLogger(logging.Handler):
    def emit(self, record):
        if "POST" in str(record.msg) and "chat/completions" in str(record.msg):
            print("🔍 CAPTURED LLM REQUEST")
            # The actual body is in the httpcore logs

logging.getLogger("httpcore").addHandler(RequestLogger())

# Now create your agent and send a message
# client = Letta(base_url="http://localhost:8283")
# agent = client.agents.create(...)

# Send test message
response = client.agents.messages.create(
    agent_id=agent.id,
    messages=[{"role": "user", "content": [{"type": "text", "text": "Hello"}]}]
)

# Check the debug output for the system prompt sent to OpenRouter

httpcore.connection - DEBUG - connect_tcp.started host='localhost' port=8283 local_address=None timeout=60 socket_options=None
httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7ff65b12c510>
httpcore.http11 - DEBUG - send_request_headers.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - send_request_headers.complete
httpcore.http11 - DEBUG - send_request_body.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - send_request_body.complete
httpcore.http11 - DEBUG - receive_response_headers.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'date', b'Tue, 11 Nov 2025 15:48:33 GMT'), (b'server', b'uvicorn'), (b'content-length', b'1263'), (b'content-type', b'application/json')])
httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/agent-6d7a65d8-ccb2-41c2-b83a-338d488b9b4e/messages "HTTP/1.1 200 OK"
httpcore.ht

In [11]:
from letta_client import Letta
import json

client = Letta(base_url="http://localhost:8283")

# Use the existing agent or create a new one
# agent = client.agents.create(...)  # If needed

# Get agent details
details = client.agents.get(agent.id)

print("="*60)
print("LETTA AGENT INSPECTION")
print("="*60)

# 1. Check all available attributes
print("\n1️⃣ Available Attributes:")
attrs = [a for a in dir(details) if not a.startswith('_')]
for attr in attrs:
    print(f"  - {attr}")

# 2. Check for system-related fields
print("\n2️⃣ System Prompt Fields:")
system_fields = ['system', 'system_prompt', 'instructions', 'prompt', 'description']
for field in system_fields:
    if hasattr(details, field):
        value = getattr(details, field)
        print(f"\n✓ {field}:")
        print(f"  {str(value)[:500]}...")  # First 500 chars

# 3. Memory blocks
print("\n3️⃣ Memory Blocks:")
if hasattr(details, 'memory'):
    if hasattr(details.memory, 'blocks'):
        for block in details.memory.blocks:
            print(f"\n  [{block.label}]")
            print(f"  {block.value}")
    else:
        print(f"  Memory: {details.memory}")

# 4. Tools
print("\n4️⃣ Available Tools:")
if hasattr(details, 'tools'):
    for tool in details.tools:
        print(f"  - {tool}")

# 5. Full JSON dump (to see everything)
print("\n5️⃣ Full Agent State (first 2000 chars):")
try:
    full_json = json.dumps(details.model_dump(), indent=2)
    print(full_json[:2000])
    print(f"\n... (total length: {len(full_json)} characters)")
except Exception as e:
    print(f"  Error: {e}")
    print(f"  Raw object: {str(details)[:500]}...")

print("\n" + "="*60)

AttributeError: 'AgentsClient' object has no attribute 'get'

In [12]:
from letta_client import Letta
import json

client = Letta(base_url="http://localhost:8283")

# Get agent details using the correct API method
try:
    details = client.get_agent(agent.id)
except AttributeError:
    # Try alternative API patterns
    try:
        details = client.agents.get_agent(agent.id)
    except:
        # List all methods to find the right one
        print("Available methods on client.agents:")
        print([m for m in dir(client.agents) if not m.startswith('_')])
        
        # Try to get via list
        all_agents = client.agents.list_agents()
        details = [a for a in all_agents if a.id == agent.id][0]

print("="*60)
print("LETTA AGENT INSPECTION")
print("="*60)

# 1. Check all available attributes
print("\n1️⃣ Available Attributes:")
attrs = [a for a in dir(details) if not a.startswith('_')]
for attr in attrs:
    print(f"  - {attr}")

# 2. Check for system-related fields
print("\n2️⃣ System Prompt Fields:")
system_fields = ['system', 'system_prompt', 'instructions', 'prompt', 'description']
for field in system_fields:
    if hasattr(details, field):
        value = getattr(details, field)
        print(f"\n✓ {field}:")
        print(f"  {str(value)[:500]}...")  # First 500 chars

# 3. Memory blocks
print("\n3️⃣ Memory Blocks:")
if hasattr(details, 'memory'):
    if hasattr(details.memory, 'blocks'):
        for block in details.memory.blocks:
            print(f"\n  [{block.label}]")
            print(f"  {block.value}")
    else:
        print(f"  Memory: {details.memory}")

# 4. Tools
print("\n4️⃣ Available Tools:")
if hasattr(details, 'tools'):
    for tool in details.tools:
        print(f"  - {tool}")

# 5. Full JSON dump (to see everything)
print("\n5️⃣ Full Agent State (first 2000 chars):")
try:
    full_json = json.dumps(details.model_dump(), indent=2)
    print(full_json[:2000])
    print(f"\n... (total length: {len(full_json)} characters)")
except Exception as e:
    print(f"  Error: {e}")
    # Try dict conversion instead
    try:
        full_json = json.dumps(dict(details), indent=2)
        print(full_json[:2000])
    except:
        print(f"  Raw object: {str(details)[:500]}...")

print("\n" + "="*60)

Available methods on client.agents:
['blocks', 'context', 'core_memory', 'count', 'create', 'delete', 'export_file', 'files', 'folders', 'groups', 'import_file', 'list', 'memory_variables', 'messages', 'modify', 'passages', 'retrieve', 'search', 'sources', 'templates', 'tools', 'with_raw_response']


AttributeError: 'AgentsClient' object has no attribute 'list_agents'

In [13]:
from letta_client import Letta
import json

client = Letta(base_url="http://localhost:8283")

# Use the correct API method: client.agents.retrieve()
details = client.agents.retrieve(agent.id)

print("="*60)
print("LETTA AGENT INSPECTION")
print("="*60)

# 1. Check all available attributes
print("\n1️⃣ Available Attributes:")
attrs = [a for a in dir(details) if not a.startswith('_')]
for attr in attrs:
    print(f"  - {attr}")

# 2. Check for system-related fields
print("\n2️⃣ System Prompt Fields:")
system_fields = ['system', 'system_prompt', 'instructions', 'prompt', 'description']
for field in system_fields:
    if hasattr(details, field):
        value = getattr(details, field)
        print(f"\n✓ {field}:")
        print(f"  {str(value)[:500]}...")  # First 500 chars

# 3. Memory blocks
print("\n3️⃣ Memory Blocks:")
if hasattr(details, 'memory'):
    if hasattr(details.memory, 'blocks'):
        for block in details.memory.blocks:
            print(f"\n  [{block.label}]")
            print(f"  {block.value}")
    else:
        print(f"  Memory: {details.memory}")

# 4. Tools
print("\n4️⃣ Available Tools:")
if hasattr(details, 'tools'):
    for tool in details.tools:
        print(f"  - {tool}")

# 5. Full JSON dump (to see everything)
print("\n5️⃣ Full Agent State (first 3000 chars):")
try:
    full_json = json.dumps(details.model_dump(), indent=2)
    print(full_json[:3000])
    print(f"\n... (total length: {len(full_json)} characters)")
    
    # Save to file for full inspection
    with open("/tmp/letta_agent_full.json", "w") as f:
        f.write(full_json)
    print("\n✓ Full state saved to /tmp/letta_agent_full.json")
    
except Exception as e:
    print(f"  Error: {e}")
    print(f"  Raw object: {str(details)[:1000]}...")

print("\n" + "="*60)

httpcore.connection - DEBUG - connect_tcp.started host='localhost' port=8283 local_address=None timeout=60 socket_options=None
httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7ff65b2d8490>
httpcore.http11 - DEBUG - send_request_headers.started request=<Request [b'GET']>
httpcore.http11 - DEBUG - send_request_headers.complete
httpcore.http11 - DEBUG - send_request_body.started request=<Request [b'GET']>
httpcore.http11 - DEBUG - send_request_body.complete
httpcore.http11 - DEBUG - receive_response_headers.started request=<Request [b'GET']>
httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'date', b'Tue, 11 Nov 2025 15:51:20 GMT'), (b'server', b'uvicorn'), (b'content-length', b'19472'), (b'content-type', b'application/json')])
httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-4f728c44-227e-4501-b149-ef99773db6c6 "HTTP/1.1 200 OK"
httpcore.http11 - DEBUG

In [10]:
from letta_client import Letta
import json
import os

client = Letta(base_url="http://localhost:8283")

# Create test agent
agent = client.agents.create(
    name="prompt_inspector",
    llm_config={
        "model": "openai/gpt-oss-20b",
        "model_endpoint_type": "openai",
        "model_endpoint": "https://openrouter.ai/api/v1",
        "context_window": 4096
    },
    embedding_config={
        "embedding_model": "openai/text-embedding-3-large",
        "embedding_endpoint_type": "openai",
        "embedding_endpoint": "https://openrouter.ai/api/v1",
        "embedding_dim": 1536
    },
    memory_blocks=[
        {"label": "human", "value": "Inspecting system prompt"},
        {"label": "persona", "value": "I am a test assistant"}
    ]
)

print("="*60)
print("INSPECTING LETTA AGENT")
print("="*60)

# Get full agent details
details = client.agents.get(agent.id)

print("\n🔍 Agent Fields:")
for field in dir(details):
    if not field.startswith('_'):
        print(f"  - {field}")

print("\n📝 Checking for system prompt...")
for attr in ['system', 'system_prompt', 'instructions', 'prompt']:
    if hasattr(details, attr):
        value = getattr(details, attr)
        print(f"\n✓ Found: {attr}")
        print(f"Value: {value}")

print("\n💾 Full State:")
try:
    print(json.dumps(details.model_dump(), indent=2))
except:
    print(details)

# Cleanup
client.agents.delete(agent.id)

httpcore.connection - DEBUG - connect_tcp.started host='localhost' port=8283 local_address=None timeout=60 socket_options=None
httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7ff65b1ad210>
httpcore.http11 - DEBUG - send_request_headers.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - send_request_headers.complete
httpcore.http11 - DEBUG - send_request_body.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - send_request_body.complete
httpcore.http11 - DEBUG - receive_response_headers.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'date', b'Tue, 11 Nov 2025 15:49:10 GMT'), (b'server', b'uvicorn'), (b'content-length', b'19465'), (b'content-type', b'application/json')])
httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/ "HTTP/1.1 200 OK"
httpcore.http11 - DEBUG - receive_response_body.started reque

AttributeError: 'AgentsClient' object has no attribute 'get'

In [5]:
# Add to the beginning of cell #VSC-0c55ab83
import logging
import sys

# Enable detailed HTTP logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(name)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

# Focus on httpx (what Letta uses internally)
logging.getLogger("httpx").setLevel(logging.DEBUG)

# Also enable Letta's internal logging
logging.getLogger("letta").setLevel(logging.DEBUG)

print("🔍 HTTP DEBUGGING ENABLED")
print("="*60)

# FIXED VERSION with error handling
import time

print("="*60)
print("LETTA MEMORY SYSTEM TEST (WITH ERROR HANDLING)")
print("="*60)

# Phase 1: Provide rich, structured information
print("\n📝 PHASE 1: Loading agent with information")
print("-"*60)

facts = [
    "My favorite color is ocean blue, and I love watching sunsets by the beach.",
    "I work as a software engineer at TechCorp, specializing in distributed systems.",
    "Last summer I visited Tokyo and tried authentic ramen for the first time at a small shop in Shibuya.",
    "My birthday is on March 15th, and I'm planning a hiking trip to celebrate.",
    "I have two pets: a golden retriever named Max and a tabby cat called Whiskers.",
    "I'm currently reading 'The Three-Body Problem' and loving the hard sci-fi concepts.",
    "My morning routine includes meditation, coffee with oat milk, and a 30-minute jog.",
    "I recently started learning to play the ukulele and practice every evening.",
    "My favorite cuisine is Thai food, especially green curry with jasmine rice.",
    "I'm allergic to peanuts and always carry an EpiPen with me.",
]

all_tools_used = []
failed_turns = []

for i, fact in enumerate(facts):
    print(f"\n[Turn {i+1}/10] Sending: {fact[:60]}...")
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = client.agents.messages.create(
                agent_id=agent.id,
                messages=[{
                    "role": "user",
                    "content": [{"type": "text", "text": fact}]
                }]
            )
            
            # Collect ALL tool calls from this turn
            turn_tools = []
            for msg in response.messages:
                if hasattr(msg, 'tool_call') and msg.tool_call:
                    tool_name = msg.tool_call.name
                    turn_tools.append(tool_name)
                    print(f"  🔧 Tool used: {tool_name}")
            
            if not turn_tools:
                # Check if there was a send_message (implicit tool)
                has_text_response = any(
                    hasattr(msg, 'content') and msg.content 
                    for msg in response.messages
                )
                if has_text_response:
                    print(f"  💬 Agent responded without explicit tool call (may be OK)")
                else:
                    print(f"  ⚠️ No tools used and no response")
            
            all_tools_used.extend(turn_tools)
            break  # Success, move to next fact
            
        except Exception as e:
            error_msg = str(e)
            if "No tool calls found" in error_msg:
                print(f"  ⚠️ Attempt {attempt+1}/{max_retries}: Model didn't call tools")
                if attempt < max_retries - 1:
                    print(f"     Retrying with explicit instruction...")
                    # Retry with more explicit prompt
                    fact = f"Please remember this: {fact}"
                    time.sleep(1)
                else:
                    print(f"  ❌ Failed after {max_retries} attempts")
                    failed_turns.append((i+1, fact[:60]))
                    break
            else:
                print(f"  ❌ Unexpected error: {error_msg}")
                failed_turns.append((i+1, fact[:60]))
                break
    
    time.sleep(0.5)

print(f"\n✓ Phase 1 complete. Total tool calls: {len(all_tools_used)}")
print(f"  Unique tools: {set(all_tools_used)}")
if failed_turns:
    print(f"  ⚠️ Failed turns: {len(failed_turns)}")
    for turn_num, fact_preview in failed_turns:
        print(f"    - Turn {turn_num}: {fact_preview}...")

# Phase 2: Test recall of specific facts (skip if too many failures)
if len(failed_turns) >= 5:
    print("\n⚠️ Too many failures in Phase 1, skipping recall test")
    print("   Recommendation: Use a better model (gpt-4o-mini, claude-3-5-sonnet)")
    client.agents.delete(agent.id)
    print(f"\n🗑️ Agent {agent.id} deleted")
else:
    print("\n\n🔍 PHASE 2: Testing memory recall")
    print("-"*60)
    
    recall_queries = [
        ("What is my favorite color?", "ocean blue"),
        ("Where do I work?", "TechCorp"),
        ("What city did I visit last summer?", "Tokyo"),
        ("When is my birthday?", "March 15"),
        ("What are my pets' names?", "Max"),
        ("What book am I reading?", "Three-Body Problem"),
        ("What do I drink in the morning?", "coffee"),
        ("What instrument am I learning?", "ukulele"),
        ("What is my favorite cuisine?", "Thai"),
        ("What am I allergic to?", "peanuts"),
    ]
    
    retrieval_results = []
    
    for query, expected_keyword in recall_queries:
        print(f"\n❓ Query: {query}")
        
        try:
            response = client.agents.messages.create(
                agent_id=agent.id,
                messages=[{
                    "role": "user",
                    "content": [{"type": "text", "text": query}]
                }]
            )
            
            # Track tools used for retrieval
            query_tools = []
            for msg in response.messages:
                if hasattr(msg, 'tool_call') and msg.tool_call:
                    tool_name = msg.tool_call.name
                    query_tools.append(tool_name)
                    print(f"  🔧 Tool used: {tool_name}")
            
            # Get the agent's final answer
            final_answer = ""
            for msg in reversed(response.messages):
                if hasattr(msg, 'content') and msg.content:
                    final_answer = msg.content
                    break
            
            # Check if answer contains expected keyword
            import re
            def fuzzy_match(expected, answer):
                # Normalize whitespace, hyphens, punctuation
                expected_norm = re.sub(r'[\s\-—–]+', ' ', expected.lower()).strip()
                answer_norm = re.sub(r'[\s\-—–]+', ' ', answer.lower()).strip()
                
                # Check if normalized expected is in answer
                if expected_norm in answer_norm:
                    return True
                
                # Also check if all words from expected appear in answer
                expected_words = set(expected_norm.split())
                answer_words = set(answer_norm.split())
                return expected_words.issubset(answer_words)

            contains_info = fuzzy_match(expected_keyword, final_answer)
            retrieval_results.append({
                "query": query,
                "expected": expected_keyword,
                "answer": final_answer[:100],
                "correct": contains_info,
                "tools_used": query_tools
            })
            
            status = "✓" if contains_info else "✗"
            print(f"  {status} Answer: {final_answer[:80]}...")
            
        except Exception as e:
            print(f"  ❌ Error: {str(e)[:100]}")
            retrieval_results.append({
                "query": query,
                "expected": expected_keyword,
                "answer": "ERROR",
                "correct": False,
                "tools_used": []
            })
        
        time.sleep(0.5)
    
    # Phase 3: Summary and analysis
    print("\n\n📊 PHASE 3: Test results")
    print("="*60)
    
    print(f"\n1️⃣ ALL TOOLS USED DURING TEST:")
    unique_tools = sorted(set(all_tools_used + [t for r in retrieval_results for t in r['tools_used']]))
    for tool in unique_tools:
        count = all_tools_used.count(tool) + sum(tool in r['tools_used'] for r in retrieval_results)
        print(f"   • {tool}: {count} times")
    
    print(f"\n2️⃣ MEMORY STORAGE TOOLS:")
    storage_keywords = ['insert', 'append', 'save', 'write', 'archival']
    storage_tools = [t for t in all_tools_used if any(kw in t.lower() for kw in storage_keywords)]
    print(f"   Count: {len(storage_tools)}")
    if storage_tools:
        print(f"   Examples: {set(storage_tools)}")
    else:
        print("   ⚠️  WARNING: No explicit storage tools detected!")
    
    print(f"\n3️⃣ MEMORY RETRIEVAL TOOLS:")
    retrieval_keywords = ['search', 'recall', 'retrieve', 'query', 'archival']
    retrieval_tools = [t for r in retrieval_results for t in r['tools_used'] if any(kw in t.lower() for kw in retrieval_keywords)]
    print(f"   Count: {len(retrieval_tools)}")
    if retrieval_tools:
        print(f"   Examples: {set(retrieval_tools)}")
    else:
        print("   ⚠️  WARNING: No retrieval tools used!")
    
    print(f"\n4️⃣ RECALL ACCURACY:")
    correct_recalls = sum(1 for r in retrieval_results if r['correct'])
    print(f"   {correct_recalls}/{len(recall_queries)} queries answered correctly")
    print(f"   Success rate: {correct_recalls/len(recall_queries)*100:.1f}%")
    
    print("\n5️⃣ FAILED RECALLS:")
    for r in retrieval_results:
        if not r['correct']:
            print(f"   ✗ {r['query']}")
            print(f"     Expected: '{r['expected']}' | Got: '{r['answer'][:60]}...'")
    
    print("\n6️⃣ DIAGNOSTIC:")
    if len(storage_tools) == 0:
        print("   ⚠️  Agent may not be storing information in archival memory")
        print("   → Context window might be too large (currently 2000)")
        print("   → Try reducing to 1000 or check if archival is enabled")
    elif len(retrieval_tools) == 0:
        print("   ⚠️  Agent not retrieving from memory")
        print("   → Information might be stored but not searched")
        print("   → Check if conversation_search/archival_search tools are available")
    elif correct_recalls < len(recall_queries) * 0.5:
        print("   ⚠️  Low recall accuracy despite tool usage")
        print("   → Tools may be firing but returning empty results")
        print("   → Check server logs for embedding/vector DB errors")
    else:
        print("   ✓ Memory system appears to be functioning correctly!")
    
    print("\n" + "="*60)
    print("TEST COMPLETE")
    print("="*60)
    
    # Cleanup
    client.agents.delete(agent.id)
    print(f"\n🗑️  Agent {agent.id} deleted")

🔍 HTTP DEBUGGING ENABLED
LETTA MEMORY SYSTEM TEST (WITH ERROR HANDLING)

📝 PHASE 1: Loading agent with information
------------------------------------------------------------

[Turn 1/10] Sending: My favorite color is ocean blue, and I love watching sunsets...
httpcore.connection - DEBUG - connect_tcp.started host='localhost' port=8283 local_address=None timeout=60 socket_options=None
httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7ff66d183b10>
httpcore.http11 - DEBUG - send_request_headers.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - send_request_headers.complete
httpcore.http11 - DEBUG - send_request_body.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - send_request_body.complete
httpcore.http11 - DEBUG - receive_response_headers.started request=<Request [b'POST']>
httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 400, b'Bad Request', [(b'date', b'Tue, 

KeyboardInterrupt: 

In [15]:
import os
import requests

response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
        "Content-Type": "application/json"
    },
    json={
        "model": "openai/gpt-oss-120b",
        "messages": [{"role": "user", "content": "What's the weather?"}],
        "tools": [{
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get weather",
                "parameters": {"type": "object", "properties": {}}
            }
        }]
    }
)

print(response.json())
# Check: Does response have "tool_calls" field?

{'id': 'gen-1762873029-Yp34DgAV78aqkWOXn8WJ', 'provider': 'SambaNova', 'model': 'openai/gpt-oss-120b', 'object': 'chat.completion', 'created': 1762873031, 'choices': [{'logprobs': None, 'finish_reason': 'tool_calls', 'native_finish_reason': 'tool_calls', 'index': 0, 'message': {'role': 'assistant', 'content': '', 'refusal': None, 'reasoning': 'We need to get weather using function. Use get_weather.', 'tool_calls': [{'function': {'arguments': '{}', 'name': 'get_weather'}, 'id': 'call_f976d640aa8a4a8884', 'index': 0, 'type': 'function'}], 'reasoning_details': [{'type': 'reasoning.text', 'text': 'We need to get weather using function. Use get_weather.', 'format': 'unknown', 'index': 0}]}}], 'system_fingerprint': 'fastcoe', 'usage': {'prompt_tokens': 113, 'completion_tokens': 31, 'total_tokens': 144, 'prompt_tokens_details': {'cached_tokens': 0}}}


In [16]:
# Add this as a new cell in your notebook
import letta
print(f"Letta location: {letta.__file__}")

# Also check the LLM API helpers
import letta.llm_api.helpers
print(f"Helpers location: {letta.llm_api.helpers.__file__}")

Letta location: /home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/letta/__init__.py
Helpers location: /home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/letta/llm_api/helpers.py


In [5]:
# Test if local Letta server (v0.12.1) supports letta_v1_agent
from letta_client import Letta

client = Letta(base_url="http://localhost:8283", timeout=1000)

# Try creating agent with letta_v1_agent type
print("Testing agent_type support on local Letta server...")
print("="*60)

try:
    test_agent = client.agents.create(
        name="test_agent_type_v1",
        agent_type="letta_v1_agent",  # Request letta_v1_agent
        llm_config={
            "model": "openai/gpt-oss-20b",
            "model_endpoint_type": "openai",
            "model_endpoint": "https://openrouter.ai/api/v1",
            "context_window": 16000
        },
        embedding_config={
            "embedding_model": "openai/text-embedding-3-large",
            "embedding_endpoint_type": "openai",
            "embedding_endpoint": "https://openrouter.ai/api/v1",
            "embedding_dim": 1536
        },
        memory_blocks=[
            {"label": "persona", "value": "Test persona"},
            {"label": "human", "value": ""}
        ]
    )
    
    # Retrieve and check what agent_type was actually set
    details = client.agents.retrieve(test_agent.id)
    
    print(f"✅ Agent created successfully!")
    print(f"   Requested agent_type: letta_v1_agent")
    print(f"   Actual agent_type: {details.agent_type}")
    
    if details.agent_type == "letta_v1_agent":
        print("\n🎉 Local server SUPPORTS letta_v1_agent!")
    else:
        print(f"\n⚠️ Local server converted to: {details.agent_type}")
    
    # Also check tool_rules
    if hasattr(details, 'tool_rules'):
        print(f"\n   Tool rules: {details.tool_rules}")
    
    # Cleanup
    client.agents.delete(test_agent.id)
    print(f"\n🗑️ Test agent deleted")
    
except Exception as e:
    print(f"❌ Error creating agent with letta_v1_agent: {e}")
    print("\n   This likely means local Letta v0.12.1 doesn't support this agent_type")

Testing agent_type support on local Letta server...
✅ Agent created successfully!
   Requested agent_type: letta_v1_agent
   Actual agent_type: letta_v1_agent

🎉 Local server SUPPORTS letta_v1_agent!

   Tool rules: [ContinueToolRule(tool_name='memory_insert', type='continue_loop', prompt_template=None), ContinueToolRule(tool_name='conversation_search', type='continue_loop', prompt_template=None), ContinueToolRule(tool_name='memory_replace', type='continue_loop', prompt_template=None)]

🗑️ Test agent deleted


# Test Updated Letta Agent with Custom Memory Blocks

Now let's test the agent with the new prompt structure from `src/hangman/prompts/letta_agent.py`

In [ ]:
from letta_client import Letta

# Import the prompt blocks
import sys
sys.path.append('/home/mila/b/baldelld/scratch/hangman/src')
from hangman.prompts.letta_agent import PERSONA_BLOCK, HUMAN_BLOCK

client = Letta(base_url="http://localhost:8283")

# Create agent with custom memory blocks
test_agent = client.agents.create(
    name="test_custom_prompts",
    llm_config={
        "model": "openai/gpt-oss-20b",
        "model_endpoint_type": "openai",
        "model_endpoint": "https://openrouter.ai/api/v1",
        "context_window": 4096
    },
    embedding_config={
        "embedding_model": "openai/text-embedding-3-large",
        "embedding_endpoint_type": "openai",
        "embedding_endpoint": "https://openrouter.ai/api/v1",
        "embedding_dim": 1536
    },
    memory_blocks=[
        {"label": "human", "value": HUMAN_BLOCK},
        {"label": "persona", "value": PERSONA_BLOCK}
    ]
)

print(f"✓ Agent created: {test_agent.id}")

# Retrieve and inspect
details = client.agents.retrieve(test_agent.id)

print("\n" + "="*60)
print("MEMORY BLOCKS")
print("="*60)

for block in details.memory.blocks:
    print(f"\n[{block.label.upper()}]")
    print(block.value)

print("\n" + "="*60)
print("SYSTEM PROMPT (first 500 chars)")
print("="*60)
print(details.system[:500])

# Test interaction
print("\n" + "="*60)
print("TEST INTERACTION")
print("="*60)

response = client.agents.messages.create(
    agent_id=test_agent.id,
    messages=[{"role": "user", "content": [{"type": "text", "text": "Hello! What are you designed to do?"}]}]
)

for msg in response.messages:
    if hasattr(msg, 'content') and msg.content:
        print(f"\nAgent: {msg.content}")

# Cleanup
client.agents.delete(test_agent.id)
print(f"\n✓ Agent deleted")